# Data Science Project

Anda diminta oleh pimpinan untuk membuat model prediksi klasifikasi atas `income` pada dataset **Adult Income**. Data ini diperoleh dari Machine Learning Repository UCI (https://archive.ics.uci.edu/ml/datasets/Adult).Target variabel (`income`) merupakan data categorical dengan 2 nilai `<=50k` dan `>50k`. 

Kemudian, Anda diminta untuk membandingkan model yang paling baik berdasarkan eksperimen yang Anda lakukan terhadap performa model.


Langkah pertama adalah *import library*.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import dataset
df_adult = pd.read_csv('adult.csv')

In [ ]:
df_adult.info()

Apakah kita langsung dapat menggunakan data ini secara langsung?

**Tentu Tidak !**

Kali ini, kita akan memperkenalkan tahapan-tahapan yang akan Anda lakukan sebagai data scientist. Pada umumnya, tahapan yang bisa kita lakukan setidaknya terdiri dari beberapa tahapan berikut ini:

1. Data Cleaning atau Preprocessing
2. Explorasi Data
3. Pengolahan Data
4. Evaluasi

Kali ini, kita akan melakukan tahapan 1 dan 2 terlebih dahulu.

## 1. Data Cleaning / Data Preprocessing / Data Wrangling
Tahap pertama adalah melakukan pembersihan data, tahapan ini penting karena sebagian besar pekerjaan Data Analis berkutat pada bagian ini dan tahap explorasi data. 

Di tahapan ini hal yang pertama kali dilakukan adalah melakukan *peek* data menggunakan fungsi `head`.

In [ ]:
df_adult.head()

In [ ]:
df_adult.info()

Dari data tersebut di atas, terdapat 2 jenis data yaitu `int64` dan `object`. Tipe data `object` pada umumnya dapat berupa `String` yang juga pada umumnya, data yang bersifat categorical. Dari kolom tersebut di atas, dapat kita ketahui bahwa terdapat 48.842 jenis data.

Hal yang paling penting di tahapan ini adalah:
1. Cek atas data **NULL** <br>
Data **NULL** tersebut bisa berbentuk tanda `-`, spasi, `NA` ataupun tanda lainnya. Strategi atas data **NULL** tersebut bisa kita lakukan *impute* atau pengisian, atau bisa dihilangkan seluruh baris (row) data tersebut. 
2. Cek atas data anomali (outlier) <br>
Data anomali dapat dianalisis dengan metode *standard deviasi*. Normalnya, data yang bernilai bilangan riil bisa kita cari dan hilangkan agar tidak mempengaruhi performa dari model kita.

Data anomali lainnya, bisa berupa **kesalahan ketik (typo)** ataupun nilai yang sebenarnya sama tetapi dituliskan berbeda.


Oleh karena itu, kita bisa melakukan cek satu per satu untuk kolom berikut ini:

1. `workclass`
2. `education`
3. `marital-status`
4. `occupation`
5. `relatiohship`
6. `race`
7. `gender`
8. `native-country`
9. `income`

In [ ]:
col = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender','native-country']
for c in col:
    print(c,':',df_adult[c].unique())

Namun, kita hanya akan fokus terhadap 4 kolom saja yaitu `workclass`, `marital-status`, `country`, `occupation`, karena terlihat dari data categorical di atas, ada beberapa anomali salah satunya adalah nilai `?`.

### Kolom workclass

In [ ]:
# kolom workclass
df_adult.workclass.value_counts()

Ada beberapa **anomali** pada value di dalam kolom `workclass` di antaranya, nilai `?` dan nilai `Self-emp-inc` yang sebenarnya sama dengan `Self-emp-not-inc`. 

Solusi: 
1. Lakukan perubahan pada nilai `?` menjadi `NA`, karena nilai `?` tidak dapat didefinisikan.
2. Lakukan *merge* untuk nilai kategori pada `Self-emp-inc` dan `Self-emp-not-inc`.

Lakukan cek nilai `NA` apakah ada di dalam dataset.

In [ ]:
# cek NA value di kolom workclass
df_adult.workclass.isnull().any()

Untuk mengubah nilai `?` menjadi `NA` ada **banyak** cara.


Anda jangan terpaku pada salah satu cara, bahkan Anda bisa membuat cara baru.

Sekarang, kita memiliki data **NULL** !

In [ ]:
# cek NA value di kolom workclass
df_adult.workclass.isnull().any()

Untuk melakukan merge atas nilai kategori `Self-emp-inc` dan `Self-emp-not-inc` menjadi `Self-employed` kita menggunakan metode yang sama seperti di atas. Namun, kali ini kita akan menggunakan cara yang sedikit berbeda, yaitu dengan membuat suatu fungsi baru yaitu `change_workclass`. Di sini, fungsi ini akan di-passing sebagai *argument* dalam method `apply`. Begitu juga dengan Government.

In [ ]:
def change_selfemployed(x):
    if x =='Self-emp-not-inc':
        return 'Self-employed'
    elif x =='Self-emp-inc':
        return 'Self-employed'
    else:
        return x
    
def change_government(x):
    # SILAKAN ISI DI SINI

In [ ]:
# mengganti kategori nilai untuk self employed
df_adult.workclass = df_adult.workclass.apply(change_selfemployed)

# mengganti kategori nilai untuk government


# cek atas perubahan yang sudah kita lakukan
df_adult.workclass.unique()

### Kolom Marital-status

Kita mau mengubah status pernikahan menjadi 3 saja, `Married`, `Not-married`, dan `Never-married`. 

In [ ]:
df_adult['marital-status'].value_counts()

In [ ]:
def change_married_status(x):
    # ubah Divoroced, separated dan widowed menjadi 'Not-Married'
    
    # 'Never-married' tetap
    
    # yang lain menjadi 'Married'
    

In [ ]:
df_adult['marital-status'] = df_adult['marital-status'].apply(change_married_status)
df_adult['marital-status'].unique()

### Kolom Country

Dari data `native-country` terlihat bahwa kita bisa menginginkan untuk melakukan **grouping** atas negara-negara asal tersebut ke dalam bentuk region yang sama, hal ini bertujuan untuk memberikan untuk *memberikan tambahan kekuatan* atas fitur `native-country`. Tentunya, hal ini merupakan diskresi dari masing-masing analis.

In [ ]:
df_adult['native-country'].value_counts()

Kita mengingkan adanya penggabungan antara negara dengan region yang sama. Maka, kita akan membaginya menjadi 5 bagian yaitu `north-america`, `asia`, `south-america`, `europe`, dan  `others`.

Terdapat 2 cara yang bisa digunakan, cara paling cepat untuk menghemat memory adalah sebagai berikut:
```python
# buat dictionary
dict_countries = {
    'north_america' : ["Canada", "Cuba", "Dominican-Republic", "El-Salvador", "Guatemala", "Haiti", "Honduras", 
                         "Jamaica", "Mexico", "Nicaragua","Outlying-US(Guam-USVI-etc)", "Puerto-Rico", 
                       "Trinadad&Tobago","United-States"], 
    'asia' : ["Cambodia", "China", "Hong", "India", "Iran", "Japan", "Laos",
              "Philippines", "Taiwan", "Thailand", "Vietnam"],
    'south_america ' : ["Columbia", "Ecuador", "Peru"],
    'europe' : ["England", "France", "Germany", "Greece", "Holand-Netherlands","Hungary", "Ireland", "Italy",
                    "Poland", "Portugal", "Scotland", "Yugoslavia"], 
    'others' : ["South", "?"]
    }

# buat fungsi 
def change_toregion(x):
    for k,v in dict_countries.items():
        if x in v:
            return k
        
# apply perubahan pada semua data
df_adult['native-country'] = df_adult['native-country'].apply(change_toregion)
```

Akan tetapi, kita akan menggunakan cara yang paling sederhana sebagai berikut

In [ ]:
# buat list untuk setiap region
north_america = ["Canada", "Cuba", "Dominican-Republic", "El-Salvador", "Guatemala", "Haiti", "Honduras", 
 "Jamaica", "Mexico", "Nicaragua","Outlying-US(Guam-USVI-etc)", "Puerto-Rico", 
 "Trinadad&Tobago","United-States"]
asia = ["Cambodia", "China", "Hong", "India", "Iran", "Japan", "Laos",
          "Philippines", "Taiwan", "Thailand", "Vietnam"]
south_america = ["Columbia", "Ecuador", "Peru"]
europe = ["England", "France", "Germany", "Greece", "Holand-Netherlands","Hungary", "Ireland", 
          "Italy", "Poland", "Portugal", "Scotland", "Yugoslavia"]
others = ["South", "?"]

In [ ]:
# apply perubahan satu persatu 
# north_america

# asia

# south_america

# europe

# others


In [ ]:
# perhatikan perubahannya
df_adult['native-country'].value_counts()

### Kolom Occupation

Kali ini kita hanya akan mengganti nilai `?` menjadi `NaN` dengan cara yang sama di atas.

In [ ]:
df_adult.occupation.unique()

In [ ]:
# ubah ? menjadi NaN atau np.nan


In [ ]:
df_adult.occupation.unique()

## 2. Data Exploration

Tahap yang kedua ini sangat penting untuk dilakukan, umumnya kita melihat anomali yang terjadi di data dan melakukan tindakan yang diperlukan. Selanjutnya, tahapan berikut adalah melakukan visualisasi atas data yang ada.

### Cari data anomali dan perbaiki
Saat ini kita memiliki banyak data NULL berupa `NaN`, langkah yang bisa kita lakukan adalah pengisian *(impute)* atau penghapusan. Pilihan pertama dapat kita lakukan apabila dirasa penting, namun langkah kedualah yang akan kita lakukan untuk kesederhanaan.

In [ ]:
len(df_adult[df_adult.occupation.isnull() == True])

In [ ]:
len(df_adult[df_adult.workclass.isnull() == True])

In [ ]:
len(df_adult[df_adult.workclass.isnull() & df_adult.occupation.isnull()])

In [ ]:
df_adult[df_adult.workclass.isnull() & df_adult.occupation.isnull()].head()

Dapat dilihat bahwa sebanyak 2.799 data terdapat 2 nilai `NaN` pada satu baris untuk kolom `occupation` dan `workclass`. Sedangkan, 2.809 hanya `occupation`.

Karena kita ingin memprediksi besaran `income` maka, kita harus memiliki nilai ini. Oleh karena itu, kita akan menghilangkan semua data `NaN` ini.

In [ ]:
# Hapus record yang memiliki nilai NaN pada attributnya
df_adult = df_adult.dropna(how='any')

In [ ]:
len(df_adult)

Sekarang, kita hanya memiliki 46.033 dataset setelah dibersihkan.

### Cari Informasi Sebanyak-banyaknya dari data
Tahapan ini dapat kita sebut dengan **Data Exploration**, salah satu tools yang kita gunakan adalah **visualisasi**. Teknik ini menjadi penting karena kita bisa mendapatkan informasi yang sebelumnya tidak bisa didapatkan hanya dari angka dan kata.

Berikut ini kita tampilkan plot informasi pada setiap kolom terhadap income

In [ ]:
from math import ceil

fig = plt.figure(figsize=(20,15))
cols = 5
rows = ceil(float(df_adult.shape[1]) / cols)
# looping setiap kolom dan indeksnya
for i, column in enumerate(df_adult.columns):
    # urutan setiap subplot
    ax = fig.add_subplot(rows, cols, i + 1)
    # set judul chart
    ax.set_title(column)
    
    if df_adult.dtypes[column] == np.object:
        df_adult[column].value_counts().plot(kind="bar", axes=ax)
    else:
        df_adult[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
        
plt.subplots_adjust(hspace=0.7, wspace=0.2)

Dapat dilihat dari data di atas, sebagian besar data ada di daerah `north_america`, dan sebagian besar income di bawah $50,000. Di samping itu, ras yang mendominasi adalah kulit putih dan gendernya pria.


In [ ]:
plt.figure(figsize=(16, 8))
plt.hist(df_adult.age, bins=100, normed=None, histtype='bar', stacked=True)
plt.show()


Berikutnya kita akan melakukan visualisasi dengan menggunakan *heatmap*, unutk mendapatkan korelasi antara variabel. Akan tetapi, sebelumnya kita akan melakukan `label encoding` pada target variabel dan `gender`, karena mereka adalah binary variabel.

In [ ]:
salary_map={'<=50K': 1,'>50K':0}

df_adult.income = df_adult['income'].map(salary_map).astype(int)

df_adult['gender'] = df_adult['gender'].map({'Male':1,'Female':0}).astype(int)


Di samping itu, terdapat data categorical yaitu `education` yang akan kita lakukan encoding, karena terlihat sangat mirip dengan `educational-num`. Untuk itu, kita coba lakukan `LabelEncoder` pada fitur ini.

In [ ]:
df_adult.education.unique()

In [1]:
from sklearn.preprocessing import LabelEncoder

# Lakukan Label Encoder pada kolom education


NameError: name 'df_adult' is not defined

### Visualisasi heatmap untuk mencari relasi variabel
Proses ini penting, terutama untuk menghilangkan variabel yang sangat berkorelasi. 

In [ ]:
def plot_correlation(df, size=15):
    corr= df.corr()
    fig, ax =plt.subplots(figsize=(size,size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)),corr.columns)
    plt.yticks(range(len(corr.columns)),corr.columns)
    plt.show()

plot_correlation(df_adult)


Dari plot di atas terlihat dengan jelas bahwa `education` dan `educational-num` memang sangat berkaitan. Maka kita akan buang salah satu, tetapi `educational-num` ternyata memiliki properti **ordinal**, yaitu berurutan dengan makna dimana angka lebih besar, memiliki tingkat edukasi yang lebih tinggi.

In [ ]:
del df_adult['education']

### Encode Categorical Data
Tahapan ini merupakan tahap akhir sebelum dilakukan data analisis.

Seluruh fitur data yang bersifat categorical string, harus diubah menjadi angka. Terdapat 2 cara yang umum dilakukan, yaitu cara manual dengan mengaplikasikan fungsi `replace`, `map`, `apply` dan lainnya, serta cara lain yaitu menggunakan fungsi `LabelEncoder`. 

In [ ]:
df_adult.columns

In [ ]:
# Lakukan Label Encoder pada 
# native-country

# marital-status

# relationship

# race

# occupation

# workclass


## 3. Data Analisis
Tahap ini merupakan tahapan pembentukan model dari dataset yang ada. 
Kali ini, Anda akan diminta untuk melakukan *model selection* yang artinya, melakukan pemilihan model yang paling bagus di antara model-model yang ada. 

Sebagaimana biasa, kita akan melakukan `train_test_split`. Kali ini, kita akan memisahkan antara data training dan data validasi.

In [ ]:
df = df_adult.copy()

In [ ]:
from sklearn.model_selection import train_test_split

# X merupakan dataset kecuali target variabel
X = df.drop(['income'],axis=1)
y = df.income

# kita gunakan 70% data training
split_size=0.3

# Creation of Train and Test dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=split_size,random_state=22)

# Creation of Train and validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=5)

In [ ]:
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Lakukan Pembuatan model dengan menggunakan `RandomForestClassifier`

In [ ]:
# import library

# buat objek model

# fit model

# panggil fungsi predict
RF_prediction = ....

Buat evaluasi atas model `RandomForestClassifier`.

In [ ]:
print ('-'*40)
print ('Accuracy score:')
print (accuracy_score(y_test, RF_prediction))
print ('-'*40)
print ('Confusion Matrix:')
print (confusion_matrix(y_test, RF_prediction))
print ('-'*40)
print ('Classification Matrix:')
print (classification_report(y_test, RF_prediction))

Lakukan Pembuatan model dengan menggunakan `SVC`

In [ ]:
# import library

# buat objek model

# fit model

# panggil fungsi predict
svc_prediction = ....

Buat evaluasi atas model `SVC`.

In [ ]:
print ('-'*40)
print ('Accuracy score:')
print (accuracy_score(y_test,svc_prediction))
print ('-'*40)
print ('Confusion Matrix:')
print (confusion_matrix(y_test,svc_prediction))
print ('-'*40)
print ('Classification Matrix:')
print (classification_report(y_test,svc_prediction))